# 🏁 RaceTagger Scene Classifier Training - Google Colab

Training ottimizzato per scene classification con 3 architetture:
- **MobileNetV3-Small** (2.5M params) - Lightweight, target <6MB
- **ResNet18** (11.5M params) - Replicare risultati Roboflow (89%)
- **EfficientNet-B0** (5.3M params) - Compromesso size/accuracy

## Setup
1. Runtime → Change runtime type → GPU (T4)
2. Upload dataset to Google Drive or use upload cell
3. Run cells sequentially

## Dataset Structure Expected
```
f1_scenes_dataset/
├── processed/
│   ├── train/
│   │   ├── racing_action/
│   │   ├── portrait_paddock/
│   │   ├── podium_celebration/
│   │   ├── garage_pitlane/
│   │   └── crowd_scene/
│   ├── val/
│   └── test/
```


In [ ]:
# Check GPU availability
!nvidia-smi

# Install dependencies
!pip install -q tensorflow pillow tensorflowjs matplotlib seaborn

import tensorflow as tf
print(f"\n✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Mount Google Drive
from google.colab import drive
import os
from pathlib import Path

drive.mount('/content/drive')

print("\n🔍 Searching for dataset on Google Drive...\n")

# Possible dataset locations on Drive
POSSIBLE_PATHS = [
    '/content/drive/MyDrive/f1_scenes_dataset',
    '/content/drive/MyDrive/RaceTagger/f1_scenes_dataset',
    '/content/drive/MyDrive/ml-training/f1_scenes_dataset',
]

# Find dataset
DATASET_ROOT = None
for path in POSSIBLE_PATHS:
    if os.path.exists(path):
        DATASET_ROOT = path
        print(f"✅ Dataset found: {path}")
        break

if DATASET_ROOT is None:
    print("❌ Dataset not found in common locations!")
    print("\n📂 Available folders in MyDrive:")
    !ls -la /content/drive/MyDrive/
    print("\n⚠️  Please upload f1_scenes_dataset to MyDrive root")
    print("   Expected structure:")
    print("   MyDrive/")
    print("   └── f1_scenes_dataset/")
    print("       └── processed/")
    print("           ├── train/")
    print("           ├── val/")
    print("           └── test/")
    raise FileNotFoundError("Dataset not found on Google Drive")

# Set processed dataset path
DATASET_PATH = os.path.join(DATASET_ROOT, 'processed')

# Verify structure
required_dirs = ['train', 'val', 'test']
for subdir in required_dirs:
    subdir_path = os.path.join(DATASET_PATH, subdir)
    if not os.path.exists(subdir_path):
        print(f"❌ Missing directory: {subdir}")
        raise FileNotFoundError(f"Expected directory not found: {subdir_path}")
    
    # Check categories
    categories = os.listdir(subdir_path)
    print(f"✅ {subdir:5s}: {len(categories)} categories - {categories}")

print(f"\n✅ Dataset ready: {DATASET_PATH}")

In [ ]:
# Configuration
import os
import json
from pathlib import Path

# Training config
INPUT_SIZE = (224, 224)
BATCH_SIZE = 32
RANDOM_SEED = 42

# Categories
CATEGORIES = [
    'crowd_scene',
    'garage_pitlane', 
    'podium_celebration',
    'portrait_paddock',
    'racing_action'
]
NUM_CLASSES = len(CATEGORIES)

# Model-specific configs
CONFIGS = {
    'mobilenet': {
        'phase1_epochs': 20,
        'phase1_lr': 1e-3,
        'phase2_epochs': 40,  # Increased from 30
        'phase2_lr': 5e-5,    # More conservative
        'unfreeze_layers': 30,  # More layers
        'dense_units': 256,
        'dropout': 0.3
    },
    'resnet18': {
        'phase1_epochs': 15,
        'phase1_lr': 1e-3,
        'phase2_epochs': 30,
        'phase2_lr': 1e-4,
        'unfreeze_layers': 40,
        'dense_units': 512,
        'dropout': 0.4
    },
    'efficientnet': {
        'phase1_epochs': 15,
        'phase1_lr': 1e-3,
        'phase2_epochs': 35,
        'phase2_lr': 1e-4,
        'unfreeze_layers': 30,
        'dense_units': 384,
        'dropout': 0.35
    }
}

# Data augmentation
ROTATION_RANGE = 15
WIDTH_SHIFT = 0.1
HEIGHT_SHIFT = 0.1
ZOOM_RANGE = 0.15
BRIGHTNESS_RANGE = (0.8, 1.2)

print("✅ Configuration loaded")

In [ ]:
# Data loading utilities
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from typing import Tuple, Dict
import numpy as np

def compute_class_weights(train_dir: Path) -> Dict[int, float]:
    """Compute class weights for imbalanced datasets"""
    class_counts = {}
    
    for i, category in enumerate(sorted(CATEGORIES)):
        category_dir = train_dir / category
        if category_dir.exists():
            n_images = len(list(category_dir.glob('*.jpg')))
            class_counts[i] = n_images
    
    total_images = sum(class_counts.values())
    n_classes = len(class_counts)
    
    # Inverse frequency weighting
    class_weights = {}
    for i, count in class_counts.items():
        weight = total_images / (n_classes * count)
        class_weights[i] = weight
    
    return class_weights

def create_data_generators() -> Tuple[ImageDataGenerator, ImageDataGenerator]:
    """Create training and validation data generators"""
    # Training augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=ROTATION_RANGE,
        width_shift_range=WIDTH_SHIFT,
        height_shift_range=HEIGHT_SHIFT,
        zoom_range=ZOOM_RANGE,
        horizontal_flip=True,
        brightness_range=BRIGHTNESS_RANGE,
        fill_mode='nearest'
    )
    
    # Validation (only rescaling)
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    return train_datagen, val_datagen

def load_datasets(train_datagen, val_datagen, dataset_path):
    """Load train and validation datasets"""
    train_path = Path(dataset_path) / 'train'
    val_path = Path(dataset_path) / 'val'
    
    # Training set
    train_generator = train_datagen.flow_from_directory(
        str(train_path),
        target_size=INPUT_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True,
        seed=RANDOM_SEED
    )
    
    # Validation set
    val_generator = val_datagen.flow_from_directory(
        str(val_path),
        target_size=INPUT_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, val_generator

# Load data
train_datagen, val_datagen = create_data_generators()
train_gen, val_gen = load_datasets(train_datagen, val_datagen, DATASET_PATH)

# Compute class weights
class_weights = compute_class_weights(Path(DATASET_PATH) / 'train')

print(f"\n📊 Dataset Info:")
print(f"  Train samples: {train_gen.samples}")
print(f"  Val samples: {val_gen.samples}")
print(f"  Classes: {CATEGORIES}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"\n⚖️  Class weights: {class_weights}")

In [ ]:
# Model building functions
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import (
    MobileNetV3Small,
    ResNet50,  # Colab might not have ResNet18, we'll use ResNet50 with fewer layers
    EfficientNetB0
)

def build_mobilenet_model(num_classes: int, config: dict, freeze_base: bool = True):
    """
    Build MobileNetV3-Small with FIXED training parameter.
    BUG FIX: Removed training=False to allow batch norm updates during fine-tuning.
    """
    print("\n🏗️  Building MobileNetV3-Small...")
    
    base_model = MobileNetV3Small(
        input_shape=(*INPUT_SIZE, 3),
        include_top=False,
        weights='imagenet',
        minimalistic=False
    )
    
    base_model.trainable = not freeze_base
    
    # Build model
    inputs = keras.Input(shape=(*INPUT_SIZE, 3))
    
    # BUG FIX: Remove training=False! This was blocking batch norm updates
    x = base_model(inputs)  # Let Keras handle training mode automatically
    
    x = layers.GlobalAveragePooling2D(name='global_avg_pool')(x)
    x = layers.Dense(config['dense_units'], activation='relu', name='dense_classifier')(x)
    x = layers.Dropout(config['dropout'], name='dropout')(x)
    outputs = layers.Dense(num_classes, activation='softmax', name='predictions')(x)
    
    model = keras.Model(inputs, outputs, name='mobilenet_scene_classifier')
    
    print(f"  Total parameters: {model.count_params():,}")
    print(f"  Base trainable: {not freeze_base}")
    
    return model

def build_resnet_model(num_classes: int, config: dict, freeze_base: bool = True):
    """Build ResNet50 (simulating ResNet18 with fewer unfrozen layers)"""
    print("\n🏗️  Building ResNet50 (ResNet18-like config)...")
    
    base_model = ResNet50(
        input_shape=(*INPUT_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    
    base_model.trainable = not freeze_base
    
    inputs = keras.Input(shape=(*INPUT_SIZE, 3))
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(config['dense_units'], activation='relu')(x)
    x = layers.Dropout(config['dropout'])(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs, name='resnet_scene_classifier')
    
    print(f"  Total parameters: {model.count_params():,}")
    
    return model

def build_efficientnet_model(num_classes: int, config: dict, freeze_base: bool = True):
    """Build EfficientNet-B0"""
    print("\n🏗️  Building EfficientNet-B0...")
    
    base_model = EfficientNetB0(
        input_shape=(*INPUT_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    
    base_model.trainable = not freeze_base
    
    inputs = keras.Input(shape=(*INPUT_SIZE, 3))
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(config['dense_units'], activation='relu')(x)
    x = layers.Dropout(config['dropout'])(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs, name='efficientnet_scene_classifier')
    
    print(f"  Total parameters: {model.count_params():,}")
    
    return model

def unfreeze_top_layers(model, n_layers: int):
    """Unfreeze top N layers of base model for fine-tuning"""
    base_model = model.layers[1]  # Base model is second layer
    
    base_model.trainable = True
    for layer in base_model.layers[:-n_layers]:
        layer.trainable = False
    
    trainable_count = sum([1 for layer in base_model.layers if layer.trainable])
    print(f"\n🔓 Unfrozen top {trainable_count}/{len(base_model.layers)} layers")

print("✅ Model builders ready")

In [ ]:
# Training utilities
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    ReduceLROnPlateau,
    TensorBoard,
    Callback
)
from datetime import datetime

def create_callbacks(model_name: str, phase: str):
    """Create training callbacks"""
    callbacks = []
    
    # Model checkpoint
    checkpoint_dir = Path(f'/content/checkpoints/{model_name}')
    checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    checkpoint_cb = ModelCheckpoint(
        str(checkpoint_dir / f'{phase}_best.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
    callbacks.append(checkpoint_cb)
    
    # Early stopping
    early_stop_cb = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )
    callbacks.append(early_stop_cb)
    
    # Reduce learning rate
    reduce_lr_cb = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    )
    callbacks.append(reduce_lr_cb)
    
    return callbacks

def train_two_phase(model, config: dict, model_name: str):
    """Two-phase transfer learning: freeze base, then fine-tune"""
    results = {}
    
    # ─────────────────────────────────────────────
    # PHASE 1: Train classification head
    # ─────────────────────────────────────────────
    print("\n" + "="*60)
    print(f"🔒 PHASE 1: Train Classification Head - {model_name}")
    print("="*60)
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config['phase1_lr']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    phase1_callbacks = create_callbacks(model_name, 'phase1')
    
    history_phase1 = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=config['phase1_epochs'],
        callbacks=phase1_callbacks,
        class_weight=class_weights,
        verbose=1
    )
    
    results['phase1'] = history_phase1.history
    
    # ─────────────────────────────────────────────
    # PHASE 2: Fine-tune top layers
    # ─────────────────────────────────────────────
    print("\n" + "="*60)
    print(f"🔓 PHASE 2: Fine-tune Top Layers - {model_name}")
    print("="*60)
    
    unfreeze_top_layers(model, config['unfreeze_layers'])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config['phase2_lr']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    phase2_callbacks = create_callbacks(model_name, 'phase2')
    
    history_phase2 = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=config['phase2_epochs'],
        callbacks=phase2_callbacks,
        class_weight=class_weights,
        verbose=1
    )
    
    results['phase2'] = history_phase2.history
    
    return results

print("✅ Training utilities ready")

## 📱 Train MobileNetV3-Small (Fixed)

Lightweight model with BUG FIX for `training=False` issue.

**Target:**
- Accuracy: 75-80%
- Size: 2-3MB after quantization
- Inference: <50ms


In [ ]:
# Train MobileNetV3-Small
mobilenet_model = build_mobilenet_model(
    num_classes=NUM_CLASSES,
    config=CONFIGS['mobilenet'],
    freeze_base=True
)

mobilenet_results = train_two_phase(
    mobilenet_model,
    CONFIGS['mobilenet'],
    'mobilenet'
)

# Save final model
mobilenet_model.save('/content/mobilenet_scene_classifier_final.keras')

# Print results
final_val_acc = mobilenet_results['phase2']['val_accuracy'][-1]
print(f"\n✅ MobileNetV3 Final Validation Accuracy: {final_val_acc:.4f}")

## 🏗️ Train ResNet50 (ResNet18-like config)

More powerful model to replicate Roboflow results (89% accuracy).

**Target:**
- Accuracy: 85-92%
- Size: 10-15MB after quantization
- Inference: 80-120ms


In [ ]:
# Train ResNet
resnet_model = build_resnet_model(
    num_classes=NUM_CLASSES,
    config=CONFIGS['resnet18'],
    freeze_base=True
)

resnet_results = train_two_phase(
    resnet_model,
    CONFIGS['resnet18'],
    'resnet'
)

# Save final model
resnet_model.save('/content/resnet_scene_classifier_final.keras')

# Print results
final_val_acc = resnet_results['phase2']['val_accuracy'][-1]
print(f"\n✅ ResNet Final Validation Accuracy: {final_val_acc:.4f}")

## ⚡ Train EfficientNet-B0

Balanced model - compromise between size and accuracy.

**Target:**
- Accuracy: 80-88%
- Size: 4-6MB after quantization
- Inference: 50-80ms


In [ ]:
# Train EfficientNet
efficientnet_model = build_efficientnet_model(
    num_classes=NUM_CLASSES,
    config=CONFIGS['efficientnet'],
    freeze_base=True
)

efficientnet_results = train_two_phase(
    efficientnet_model,
    CONFIGS['efficientnet'],
    'efficientnet'
)

# Save final model
efficientnet_model.save('/content/efficientnet_scene_classifier_final.keras')

# Print results
final_val_acc = efficientnet_results['phase2']['val_accuracy'][-1]
print(f"\n✅ EfficientNet Final Validation Accuracy: {final_val_acc:.4f}")

## 📊 Model Comparison

Compare all trained models and select the best one.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Collect results
comparison = {
    'Model': ['MobileNetV3-Small', 'ResNet50', 'EfficientNet-B0'],
    'Val Accuracy': [
        mobilenet_results['phase2']['val_accuracy'][-1],
        resnet_results['phase2']['val_accuracy'][-1],
        efficientnet_results['phase2']['val_accuracy'][-1]
    ],
    'Val Loss': [
        mobilenet_results['phase2']['val_loss'][-1],
        resnet_results['phase2']['val_loss'][-1],
        efficientnet_results['phase2']['val_loss'][-1]
    ],
    'Parameters (M)': [
        mobilenet_model.count_params() / 1e6,
        resnet_model.count_params() / 1e6,
        efficientnet_model.count_params() / 1e6
    ]
}

df = pd.DataFrame(comparison)
df = df.sort_values('Val Accuracy', ascending=False)

print("\n" + "="*60)
print("📊 MODEL COMPARISON RESULTS")
print("="*60)
print(df.to_string(index=False))

# Find best model
best_model_name = df.iloc[0]['Model']
best_accuracy = df.iloc[0]['Val Accuracy']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Accuracy: {best_accuracy:.4f}")

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.bar(df['Model'], df['Val Accuracy'])
ax1.set_ylabel('Validation Accuracy')
ax1.set_title('Model Accuracy Comparison')
ax1.axhline(y=0.88, color='r', linestyle='--', label='Target (88%)')
ax1.legend()
ax1.set_ylim(0, 1)

ax2.bar(df['Model'], df['Parameters (M)'])
ax2.set_ylabel('Parameters (Millions)')
ax2.set_title('Model Size Comparison')

plt.tight_layout()
plt.savefig('/content/model_comparison.png', dpi=150)
plt.show()

print("\n✅ Comparison complete!")

## 📦 Export to TensorFlow.js

Convert best model to TensorFlow.js format for browser/Electron deployment.


In [ ]:
import tensorflowjs as tfjs

# Select best model for export
# Manually set or use comparison results
best_model_file = '/content/resnet_scene_classifier_final.keras'  # Change based on results
best_model_name = 'resnet'  # Change based on results

# Load best model
best_model = keras.models.load_model(best_model_file)

# Export to TensorFlow.js
tfjs_output_dir = f'/content/tfjs_models/{best_model_name}'
tfjs.converters.save_keras_model(best_model, tfjs_output_dir)

print(f"\n✅ Model exported to: {tfjs_output_dir}")

# Export quantized version (int8)
tfjs_quantized_dir = f'/content/tfjs_models/{best_model_name}_quantized'
tfjs.converters.save_keras_model(
    best_model,
    tfjs_quantized_dir,
    quantization_dtype_map={'uint8': '*'}  # Quantize all layers
)

print(f"✅ Quantized model exported to: {tfjs_quantized_dir}")

# Save class labels
class_labels = {
    'categories': CATEGORIES,
    'category_to_index': {cat: i for i, cat in enumerate(CATEGORIES)},
    'index_to_category': {i: cat for i, cat in enumerate(CATEGORIES)},
    'input_size': INPUT_SIZE,
    'num_classes': NUM_CLASSES
}

with open(f'{tfjs_output_dir}/class_labels.json', 'w') as f:
    json.dump(class_labels, f, indent=2)

with open(f'{tfjs_quantized_dir}/class_labels.json', 'w') as f:
    json.dump(class_labels, f, indent=2)

print("\n✅ Export complete!")
print("\n📦 Download files:")
print(f"   - {tfjs_output_dir}/")
print(f"   - {tfjs_quantized_dir}/")

# Create zip for easy download
!zip -r /content/scene_classifier_tfjs.zip /content/tfjs_models/
print("\n✅ Created zip: /content/scene_classifier_tfjs.zip")

## 🧪 Test Model on Sample Images

Validate exported model on test set.


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load test images
test_dir = Path(DATASET_PATH) / 'test'

# Test on random images from each category
print("\n🧪 Testing model on sample images...\n")

for category in CATEGORIES:
    category_dir = test_dir / category
    if not category_dir.exists():
        continue
    
    test_images = list(category_dir.glob('*.jpg'))[:3]  # Test 3 images per category
    
    for img_path in test_images:
        # Load and preprocess image
        img = image.load_img(img_path, target_size=INPUT_SIZE)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0
        
        # Predict
        predictions = best_model.predict(img_array, verbose=0)
        predicted_class = CATEGORIES[np.argmax(predictions[0])]
        confidence = np.max(predictions[0])
        
        # Check if correct
        correct = "✅" if predicted_class == category else "❌"
        
        print(f"{correct} True: {category:20s} | Predicted: {predicted_class:20s} | Confidence: {confidence:.2%}")

print("\n✅ Testing complete!")

## 📥 Download Models

Download trained models and TensorFlow.js exports to your local machine.


In [ ]:
from google.colab import files

# Download options
print("\n📥 Available downloads:\n")
print("1. TensorFlow.js models (all formats): scene_classifier_tfjs.zip")
print("2. Best Keras model: best_scene_classifier.keras")
print("3. Training history: training_results.json")

# Save training results
training_summary = {
    'mobilenet': {
        'final_val_acc': float(mobilenet_results['phase2']['val_accuracy'][-1]),
        'final_val_loss': float(mobilenet_results['phase2']['val_loss'][-1]),
        'parameters': int(mobilenet_model.count_params())
    },
    'resnet': {
        'final_val_acc': float(resnet_results['phase2']['val_accuracy'][-1]),
        'final_val_loss': float(resnet_results['phase2']['val_loss'][-1]),
        'parameters': int(resnet_model.count_params())
    },
    'efficientnet': {
        'final_val_acc': float(efficientnet_results['phase2']['val_accuracy'][-1]),
        'final_val_loss': float(efficientnet_results['phase2']['val_loss'][-1]),
        'parameters': int(efficientnet_model.count_params())
    },
    'best_model': best_model_name,
    'dataset_info': {
        'train_samples': train_gen.samples,
        'val_samples': val_gen.samples,
        'categories': CATEGORIES
    }
}

with open('/content/training_results.json', 'w') as f:
    json.dump(training_summary, f, indent=2)

# Download files
print("\n🚀 Starting downloads...\n")

# TensorFlow.js zip
files.download('/content/scene_classifier_tfjs.zip')

# Best model
files.download(best_model_file)

# Training results
files.download('/content/training_results.json')

# Model comparison plot
files.download('/content/model_comparison.png')

print("\n✅ All downloads complete!")
print("\n📋 Next steps:")
print("1. Extract scene_classifier_tfjs.zip")
print("2. Copy TensorFlow.js model to RaceTagger project")
print("3. Test inference performance in Electron app")
print("4. Compare quantized vs non-quantized model sizes and speeds")